### Imports

In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import datetime
import sqlite3

from textblob import TextBlob
import altair as alt

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

### Set Up Reddit Instance

In [3]:
# access password and client secret id via local files
with open('pw.txt', 'r') as file1:
    pw = file1.read()

with open('client_secret.txt', 'r') as file2:
    cs = file2.read()

# create a praw Reddit instance with app credentials and secret info passed through
reddit = praw.Reddit(
    client_id="XbesrQBvKymjgLdgg_D6lA",
    client_secret=cs,
    user_agent="NFLTextAnalysis/0.0.1",
    username="ta_api",
    password=pw
)

# store an instance of the Redditor object for the gamethread account
nfl_bot = reddit.redditor("nfl_gamethread")

### Get Raw Data

In [4]:
gamethread_list = nfl_gamethreads.get_submissions(nfl_bot, 1000)

### Transform in Gamethread DataFrame

In [20]:
gamethread_df = nfl_gamethreads.build_gamethread_df(reddit, gamethread_list, season=2021)

### Store Data in CSV & SQL

In [27]:
gamethread_df.rename(inplace=True, columns={'predicted_diff':'predicted_difference'})

In [ ]:
gamethread_df.to_csv("gamethreads_2021.csv", index=False)

In [28]:
with sqlite3.connect("nfl_gamethreads.db") as conn:
    gamethread_df.to_sql(name='gamethreads', con=conn, if_exists='append', index=False)

## Get Comments

In [ ]:
week_4_comments = nfl_gamethreads.build_weekly_comments_df(gamethread_df=gamethread_df, week=4, reddit=reddit)

In [ ]:
sub = gamethread_df[gamethread_df['week']==3]['id'][0]

In [ ]:
week_3_comments = nfl_gamethreads.get_comments(reddit, submission_id=sub, comments_only=False)